In [1]:
from glob import glob
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import random
import numpy as np
from ats_module.TextPreprocessing import *
from ats_module.TextTagging import *
import seaborn as sns
import pickle

c:\users\choi seung un\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\choi seung un\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\users\choi seung un\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
all_df = pd.read_csv('../../data/original_20230119.csv', encoding="utf-8-sig", index_col=0)
all_df

c:\users\choi seung un\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID
0,202201,55_59,2,1,5,입금,1,펌뱅킹 입금이체,110,BC,UID13270
1,202201,60,2,1,17,입금,1,펌뱅킹 입금이체,251,BC,UID01703
2,202201,45_49,1,1,14,입금,1,펌뱅킹 입금이체,86,BC,UID04877
3,202201,45_49,2,1,16,입금,1,펌뱅킹 입금이체,17,BC,UID11157
4,202201,60,1,1,20,입금,1,펌뱅킹 입금이체,2628,BC,UID08209
...,...,...,...,...,...,...,...,...,...,...,...
4246094,202203,60,2,3,10,지급,1,한전 출금이체 서비스,1000,NaN,UID07847
4246095,202203,50_54,2,3,31,지급,1,한전 출금이체 서비스,490,NaN,UID16147
4246096,202205,50_54,1,5,2,지급,1,한전 출금이체 서비스,1000,NaN,UID08421
4246097,202205,50_54,2,5,6,지급,1,한전 출금이체 서비스,1000,NaN,UID08202


In [3]:
# 적요 전처리
all_df['TEXTS_1'] = all_df['TEXTS_1'].str.replace(" ","0") # 모든 공백에 0
text = pd.read_csv('../../data/duplicates_20230125.csv', encoding="utf-8-sig", usecols=['TRANS_MD','TEXTS_1','TEXTS_2'])

text_dict = dict()
text_dict['입금'] = dict()
text_dict['지급'] = dict()

for i in tqdm(text.index):
    trans_md = text.iloc[i]['TRANS_MD']
    if trans_md == '지급':
        text_dict['지급'][text.iloc[i]['TEXTS_1']] = text.iloc[i]['TEXTS_2']
    elif trans_md == '입금':
        text_dict['입금'][text.iloc[i]['TEXTS_1']] = text.iloc[i]['TEXTS_2']
        
a = all_df[all_df['TRANS_MD']=='지급'].copy()
b = all_df[all_df['TRANS_MD']=='입금'].copy()

a['TEXTS_2'] = a['TEXTS_1'].map(text_dict['지급'])
b['TEXTS_2'] = b['TEXTS_1'].map(text_dict['입금'])

len(a[a['TEXTS_2'].isnull()]), len(b[b['TEXTS_2'].isnull()]), 


all_df = pd.concat([a,b], axis=0)
del a
del b
del text
all_df

100%|██████████████████████████████████████████████████████████████████████████| 95713/95713 [00:14<00:00, 6452.25it/s]


,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2
2029421,202201,50_54,1,1,25,지급,1,여신이자수입,7,0000000000000(0000),UID10799,개인거래
2029422,202203,50_54,1,3,25,지급,1,여신이자수입,5,0000000000000(0000),UID05521,개인거래
2029423,202204,60,1,4,18,지급,1,여신이자수입,54,0000000000000(0000),UID14826,개인거래
2029424,202201,35_39,1,1,17,지급,1,여신이자수입,10,0000000000000(000),UID16384,개인거래
2029425,202201,60,1,1,10,지급,1,여신이자수입,9,0000000000000(000),UID20813,개인거래
...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,45_49,1,5,6,입금,1,한전 출금이체 서비스,0,NaN,UID14430,입금이체
2029417,202206,60,1,6,27,입금,1,한전 출금이체 서비스,0,NaN,UID18974,입금이체
2029418,202206,60,2,6,27,입금,1,한전 출금이체 서비스,0,NaN,UID04021,입금이체
2029419,202206,40_44,2,6,27,입금,1,한전 출금이체 서비스,0,NaN,UID08039,입금이체


In [4]:
# ATS_KDCD_DTL 전처리
test = all_df.copy()

test.loc[test['ATS_KDCD_DTL'].str.contains('보증료'), 'ATS_KDCD_DTL'] = "보증료"
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['외화', '해외'])), 'ATS_KDCD_DTL'] = "해외"
test.loc[test['ATS_KDCD_DTL'].str.contains('연금'), 'ATS_KDCD_DTL'] = "연금"
test.loc[test['ATS_KDCD_DTL'].str.contains('수익증권'), 'ATS_KDCD_DTL'] = "증권"
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['어음', '역환처리'])), 'ATS_KDCD_DTL'] = "어음"
test.loc[test['ATS_KDCD_DTL'].str.contains('재정자금'), 'ATS_KDCD_DTL'] = "재정자금"
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['이자'])), 'ATS_KDCD_DTL'] = "이자"

# 요구불간도 자동이체라 하는데 잘 모르겠음
# 일단 스쿨뱅킹도 자동이체쪽으로
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['타행자동이체', \
                                                     '요구불간', \
                                                     '납부자자동', \
                                                     '스쿨뱅킹', \
                                                     '일반대출'])), 'ATS_KDCD_DTL'] = "자동이체"

test.loc[test['ATS_KDCD_DTL'].str.contains('예약'), 'ATS_KDCD_DTL'] = "예약이체"
# 상여금도 급여쪽으로 일단
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['급여', '상여금', '지로대량급여'])), 'ATS_KDCD_DTL'] = "급여"
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['지로', '아파트관리비', '한전'])), 'ATS_KDCD_DTL'] = "요금납부"
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['적립식', "예금"])), 'ATS_KDCD_DTL'] = "예금"
test.loc[test['ATS_KDCD_DTL'].str.contains('대량'), 'ATS_KDCD_DTL'] = "대량이체"
# 일단 전자금융도 CMS로 잘모름
test.loc[test['ATS_KDCD_DTL'].str.contains("|".join(['CMS공동망', '전자금융', '대금', 'PG', 'AMA'])), 'ATS_KDCD_DTL'] = "전자결제"
test.loc[test['ATS_KDCD_DTL'].str.contains('펌뱅킹'), 'ATS_KDCD_DTL'] = "펌뱅킹"


index = 15
# test_result = test['ATS_KDCD_DTL'].value_counts().rename_axis('unique_values').reset_index(name='counts')[index:]
test_result = test['ATS_KDCD_DTL'].value_counts().rename_axis('unique_values').reset_index(name='counts')[:index]
print(test['ATS_KDCD_DTL'].value_counts().rename_axis('unique_values').reset_index(name='counts')[index:].sum())

ats_dict = {"펌뱅킹":1,
            "전자결제":2,
            "대량이체":3,
            "예금":4,
            "급여":5,
            "요금납부":6,
            "자동이체":7,
            "예약이체":8,
            "이자":9,
            "재정자금":10,
            "어음":11,
            "증권":12,
            "연금":13,
            "해외":14,
            "보증료":15,}

# 나머지는 그외로
test.loc[~test['ATS_KDCD_DTL'].isin(ats_dict.keys()), 'ATS_KDCD_DTL'] = "그외"
test['ATS_KDCD_DTL'].value_counts().rename_axis('unique_values').reset_index(name='counts')


ats_dict["그외"] = 0
all_df['ATS_KDCD_DTL'] = test['ATS_KDCD_DTL'].map(ats_dict)
all_df

unique_values    해외ISA 적립이체-수신팀3순위청약증거금환불대출모집인 수수료 지급및 입금-여신팀신용...
counts                                                        2235
dtype: object


,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2
2029421,202201,50_54,1,1,25,지급,1,9,7,0000000000000(0000),UID10799,개인거래
2029422,202203,50_54,1,3,25,지급,1,9,5,0000000000000(0000),UID05521,개인거래
2029423,202204,60,1,4,18,지급,1,9,54,0000000000000(0000),UID14826,개인거래
2029424,202201,35_39,1,1,17,지급,1,9,10,0000000000000(000),UID16384,개인거래
2029425,202201,60,1,1,10,지급,1,9,9,0000000000000(000),UID20813,개인거래
...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,45_49,1,5,6,입금,1,6,0,NaN,UID14430,입금이체
2029417,202206,60,1,6,27,입금,1,6,0,NaN,UID18974,입금이체
2029418,202206,60,2,6,27,입금,1,6,0,NaN,UID04021,입금이체
2029419,202206,40_44,2,6,27,입금,1,6,0,NaN,UID08039,입금이체


In [5]:
# DPS_TRN_AM 전처리

# 금액을 범주형으로 
# 0이하 = 1000원 이하
# 1이상 9이하 = 1000원이상 9000원 이하
# 10이상 49이하 = 10000원이상 49000원 이하
# 50이상 99이하 = 50000원이상 99000원 이하
# 100이상 499이하 = 100000원이상 499000원 이하
# 500이상 999이하 = 500000원이상 999000원 이하
# 1000이상 4999이하 = 1000000원이상 4999000원 이하
# 5000이상 9999이하 = 5000000원이상 9999000원 이하
# 10000이상 49900이하 = 10000000원이상 49999000원 이하
# 50000이상  = 50000000원이상

all_df['DPS_TRN_AM'] = np.digitize(all_df['DPS_TRN_AM'], bins=[0, 9, 49, 99, 499, 999, 4999, 9999, 49900]) # 구간화
all_df

# all_df['DPS_TRN_AM'] = np.log1p(test['DPS_TRN_AM'])
# all_df

,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2
2029421,202201,50_54,1,1,25,지급,1,9,1,0000000000000(0000),UID10799,개인거래
2029422,202203,50_54,1,3,25,지급,1,9,1,0000000000000(0000),UID05521,개인거래
2029423,202204,60,1,4,18,지급,1,9,3,0000000000000(0000),UID14826,개인거래
2029424,202201,35_39,1,1,17,지급,1,9,2,0000000000000(000),UID16384,개인거래
2029425,202201,60,1,1,10,지급,1,9,2,0000000000000(000),UID20813,개인거래
...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,45_49,1,5,6,입금,1,6,1,NaN,UID14430,입금이체
2029417,202206,60,1,6,27,입금,1,6,1,NaN,UID18974,입금이체
2029418,202206,60,2,6,27,입금,1,6,1,NaN,UID04021,입금이체
2029419,202206,40_44,2,6,27,입금,1,6,1,NaN,UID08039,입금이체


In [6]:
# AGE_CD 전처리
print(test['AGE_CD'].value_counts())
age_dict = {"10":0,
           "20_24":1,
           "25_29":2,
           "30_34":3,
           "35_39":4,
           "40_44":5,
           "45_49":6,
           "50_54":7,
           "55_59":8,
           "60":9,}

all_df['AGE_CD'] = test['AGE_CD'].map(age_dict)
all_df

60       722252
40_44    503163
50_54    454354
45_49    454238
35_39    442751
55_59    364528
30_34    347421
25_29    261910
20_24    148158
10        47324
Name: AGE_CD, dtype: int64


,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2
2029421,202201,7,1,1,25,지급,1,9,1,0000000000000(0000),UID10799,개인거래
2029422,202203,7,1,3,25,지급,1,9,1,0000000000000(0000),UID05521,개인거래
2029423,202204,9,1,4,18,지급,1,9,3,0000000000000(0000),UID14826,개인거래
2029424,202201,4,1,1,17,지급,1,9,2,0000000000000(000),UID16384,개인거래
2029425,202201,9,1,1,10,지급,1,9,2,0000000000000(000),UID20813,개인거래
...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,6,1,5,6,입금,1,6,1,NaN,UID14430,입금이체
2029417,202206,9,1,6,27,입금,1,6,1,NaN,UID18974,입금이체
2029418,202206,9,2,6,27,입금,1,6,1,NaN,UID04021,입금이체
2029419,202206,5,2,6,27,입금,1,6,1,NaN,UID08039,입금이체


In [7]:
# weekday 
test['day'] = test['BAS_YM'].astype(str) + test['BAS_DT'].astype(str).str.zfill(2)
test['weekday'] = pd.to_datetime(test['day'], format="%Y-%m-%d").dt.weekday

# 평일,주말 생성. 평일=0, 주말=1
test.loc[test['weekday']<5, 'weekday'] = 0
test.loc[test['weekday']>=5, 'weekday'] = 1

all_df['WEEKDAY'] = test['weekday']
all_df

,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2,WEEKDAY
2029421,202201,7,1,1,25,지급,1,9,1,0000000000000(0000),UID10799,개인거래,0
2029422,202203,7,1,3,25,지급,1,9,1,0000000000000(0000),UID05521,개인거래,0
2029423,202204,9,1,4,18,지급,1,9,3,0000000000000(0000),UID14826,개인거래,0
2029424,202201,4,1,1,17,지급,1,9,2,0000000000000(000),UID16384,개인거래,0
2029425,202201,9,1,1,10,지급,1,9,2,0000000000000(000),UID20813,개인거래,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,6,1,5,6,입금,1,6,1,NaN,UID14430,입금이체,0
2029417,202206,9,1,6,27,입금,1,6,1,NaN,UID18974,입금이체,0
2029418,202206,9,2,6,27,입금,1,6,1,NaN,UID04021,입금이체,0
2029419,202206,5,2,6,27,입금,1,6,1,NaN,UID08039,입금이체,0


In [8]:
# TRANS_MD 전처리
# # 지급: 1, 입금: 2
all_df['TRANS_MD'] = test['TRANS_MD'].map({"지급":1,"입금": 2})
all_df

,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2,WEEKDAY
2029421,202201,7,1,1,25,1,1,9,1,0000000000000(0000),UID10799,개인거래,0
2029422,202203,7,1,3,25,1,1,9,1,0000000000000(0000),UID05521,개인거래,0
2029423,202204,9,1,4,18,1,1,9,3,0000000000000(0000),UID14826,개인거래,0
2029424,202201,4,1,1,17,1,1,9,2,0000000000000(000),UID16384,개인거래,0
2029425,202201,9,1,1,10,1,1,9,2,0000000000000(000),UID20813,개인거래,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,6,1,5,6,2,1,6,1,NaN,UID14430,입금이체,0
2029417,202206,9,1,6,27,2,1,6,1,NaN,UID18974,입금이체,0
2029418,202206,9,2,6,27,2,1,6,1,NaN,UID04021,입금이체,0
2029419,202206,5,2,6,27,2,1,6,1,NaN,UID08039,입금이체,0


In [9]:
# TEXTS_2 전처리
print(test['TEXTS_2'].value_counts())
tests_dict = {"개인거래":0,
              "입금이체":1,
              "금융":2,
              "기타":3,
              "소득":4,
              "서비스이용":5,
              "주거/생활":6,
              "교육":7,
              "금융수익":8,
              "여가":9,
              "자동차":10,}

all_df['TEXTS_2'] = test['TEXTS_2'].map(tests_dict)
all_df

금융수익     1372248
금융        804438
소득        439939
개인거래      396599
주거/생활     312717
입금이체      217234
기타        141325
서비스이용      37773
여가         15881
교육          5976
자동차         1969
Name: TEXTS_2, dtype: int64


,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2,WEEKDAY
2029421,202201,7,1,1,25,1,1,9,1,0000000000000(0000),UID10799,0,0
2029422,202203,7,1,3,25,1,1,9,1,0000000000000(0000),UID05521,0,0
2029423,202204,9,1,4,18,1,1,9,3,0000000000000(0000),UID14826,0,0
2029424,202201,4,1,1,17,1,1,9,2,0000000000000(000),UID16384,0,0
2029425,202201,9,1,1,10,1,1,9,2,0000000000000(000),UID20813,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029416,202205,6,1,5,6,2,1,6,1,NaN,UID14430,1,0
2029417,202206,9,1,6,27,2,1,6,1,NaN,UID18974,1,0
2029418,202206,9,2,6,27,2,1,6,1,NaN,UID04021,1,0
2029419,202206,5,2,6,27,2,1,6,1,NaN,UID08039,1,0


In [10]:
all_df = all_df.sort_values(['BAS_YM', 'BAS_DT']).reset_index(drop=True)
use_col = ['AGE_CD', 'GENDER', 'BAS_MON', 'TRANS_MD', 'ATS_KDCD_DTL', 'DPS_TRN_AM', 'TEXTS_2', 'WEEKDAY']
data = all_df[use_col].copy()

with open("../../model/k_20230127_13.pickle","rb") as fr:
    kmodel = pickle.load(fr)

data['PREDICT'] = kmodel.predict(data)
all_df['PREDICT'] = data['PREDICT']
del data

all_df

,BAS_YM,AGE_CD,GENDER,BAS_MON,BAS_DT,TRANS_MD,CNT,ATS_KDCD_DTL,DPS_TRN_AM,TEXTS_1,UID,TEXTS_2,WEEKDAY,PREDICT
0,202201,3,1,1,1,1,1,8,2,건강최준호,UID02202,6,1,7
1,202201,3,2,1,1,1,1,8,4,광주현대아미스(주),UID01192,3,1,2
2,202201,5,2,1,1,1,1,8,3,구립용산장애인복,UID05122,3,1,3
3,202201,5,2,1,1,1,1,8,3,국민가족회비,UID08546,6,1,0
4,202201,2,2,1,1,1,1,8,4,국민김경수,UID08135,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746094,202206,4,1,6,30,2,1,1,1,NaN,UID12424,1,0,12
3746095,202206,6,1,6,30,2,1,1,1,NaN,UID19461,1,0,12
3746096,202206,7,1,6,30,2,1,1,1,NaN,UID11639,1,0,12
3746097,202206,7,2,6,30,2,1,1,1,NaN,UID06768,1,0,12


In [11]:
# test_data = all_df.sample(n=100)
# save_data = all_df[~all_df.index.isin(test_data.index)]

In [39]:
uidset = pd.pivot_table(all_df, index='UID', columns = 'PREDICT', values = 'CNT', fill_value=0, aggfunc = 'count')
uidset.columns = ["c"+str(i) for i in uidset.columns]
uidset = uidset.reset_index()
uidset

,UID,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12
0,UID00000,0,27,25,90,11,0,4,0,0,0,2,0,2
1,UID00001,10,24,3,3,0,13,19,7,13,0,50,5,27
2,UID00002,26,3,10,13,0,0,10,6,13,0,15,12,82
3,UID00003,0,85,34,43,9,5,3,0,0,7,1,0,1
4,UID00004,18,1,3,1,2,0,32,5,16,0,13,13,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21159,UID21159,0,20,37,19,11,0,26,2,1,56,0,0,0
21160,UID21160,0,62,30,30,41,3,1,0,1,5,1,0,2
21161,UID21161,0,31,79,23,4,6,0,0,3,2,0,8,1
21162,UID21162,22,1,4,21,2,0,11,7,12,0,12,12,61


In [40]:
uidset.to_csv('../../data/user_info.csv', encoding="utf-8-sig")

In [41]:
sql = 'INSERT INTO `user_info`(uid, c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, note) VALUES ("%s", %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,"%s");'
f = open("../../data/user_info.sql", "w", encoding="utf-8-sig")
query = "use real;"
f.write(query+'\n')
for index in tqdm(range(len(uidset))):
    query = sql%(uidset.iloc[index]['UID'], uidset.iloc[index]['c0'], uidset.iloc[index]['c1'], \
                 uidset.iloc[index]['c2'], uidset.iloc[index]['c3'],uidset.iloc[index]['c4'], \
                 uidset.iloc[index]['c5'], uidset.iloc[index]['c6'], uidset.iloc[index]['c7'], \
                 uidset.iloc[index]['c8'], uidset.iloc[index]['c9'], uidset.iloc[index]['c10'], \
                 uidset.iloc[index]['c11'], uidset.iloc[index]['c12'], "")

    f.write(query+'\n')
f.close()